In [2]:
#hide
from utils import *
setup_chapter()

No GPU was detected! This notebook can be *very* slow without a GPU 🐢
Using transformers v4.11.3
Using datasets v1.16.1


In [4]:
%%capture
%pip install transformers[tf,torch,sentencepiece,vision,optuna,sklearn,onnxruntime]==4.11.3

SageMaker uses an AWS IAM role to access Amazon S3 bucket, to store data, models and logs.
This role will created for you with the right access rights. In the AWS IAM service you are able to review the access.

A S3 bucket will automatically created if it not exists already.

In [5]:
import sagemaker.huggingface
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::803492327198:role/service-role/AmazonSageMaker-
ExecutionRole-20210928T145176
sagemaker bucket: sagemaker-eu-central-1-803492327198
sagemaker session region: eu-central-1


We use this function to create a Hugging Face Model Class, where we are going to download the model from the Hugging Face hub.
This class would also allow to use a trained model stored on the Amazon S3 bucket. 

Next, an endpoint will be created and this endpoint will host your model.
Based on the model requirements you can choose a specific instance type which are equipped differently in memory, cpu, gpu.
There are different inferences available, such as real-time, asynchronous or serverless.
If you are not sure which inference works best for your model, you use Amazon SageMaker Inference Recommender.
https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html 


To view all options see the documentation: https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/index.html 

Depending on Transformer version, PyTorch/TensorFlow version and Python version, the mapping for the Hugging Face Model Class can be found here: https://huggingface.co/docs/sagemaker/reference#inference-dlc-overview 

In [ ]:
from sagemaker.huggingface.model import HuggingFaceModel

def setup_endpoint(model_name, task_name):
    # Hub Model configuration. <https://huggingface.co/models>
    hub = {
      'HF_MODEL_ID': model_name, # model_id from hf.co/models
      'HF_TASK': task_name # NLP task you want to use for predictions
    }

    # create Hugging Face Model Class
    huggingface_model = HuggingFaceModel(
       env=hub, # configuration for loading model from Hub
       role=role, # iam role with permissions to create an Endpoint
       transformers_version="4.17.0", # transformers version used
       pytorch_version="1.10.2", # pytorch version used
       py_version="py38" # python version used
    )

    # deploy model to SageMaker Inference
    predictor = huggingface_model.deploy(
       initial_instance_count=1, # how many instances used
       instance_type="ml.m5.xlarge" # instance type
    )
    return predictor

# Hello Transformers

## The Encoder-Decoder Framework

## Attention Mechanisms

## Transfer Learning in NLP

## Hugging Face Transformers: Bridging the Gap

## A Tour of Transformer Applications

In [10]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

### Text Classification

In [ ]:
predictor = setup_endpoint('distilbert-base-uncased-finetuned-sst-2-english', 'text-classification')

In [28]:
# example request, you always need to define "inputs"
import pandas as pd

# request
outputs = predictor.predict({"inputs": text})
pd.DataFrame(outputs)    

,label,score
0,NEGATIVE,0.901545


In [31]:
predictor.delete_endpoint()

### Named Entity Recognition

In [43]:
predictor = setup_endpoint("dbmdz/bert-large-cased-finetuned-conll03-english", "ner")

------!

In [46]:
outputs = predictor.predict({"inputs": text, "parameters": {"aggregation_strategy": "simple"}})
pd.DataFrame(outputs)    

,entity_group,score,word,start,end
0,ORG,0.879010,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97
3,MISC,0.556570,Mega,208,212
4,PER,0.590256,##tron,212,216
5,ORG,0.669692,Decept,253,259
6,MISC,0.498349,##icons,259,264
7,MISC,0.775362,Megatron,350,358
8,MISC,0.987854,Optimus Prime,367,380
9,PER,0.812096,Bumblebee,502,511


In [48]:
predictor.delete_endpoint()

### Question Answering 

In [50]:
predictor = setup_endpoint("distilbert-base-cased-distilled-squad", 'question-answering')

-----!

In [51]:
question = "What does the customer want?"

outputs = predictor.predict({"inputs": {
    "question": question,
    "context": text
    }
})

pd.DataFrame([outputs])    

,score,start,end,answer
0,0.631292,335,358,an exchange of Megatron


In [ ]:
predictor.delete_endpoint()

### Summarization

In [53]:
predictor = setup_endpoint("sshleifer/distilbart-cnn-12-6", 'summarization')

-----!

In [55]:
outputs = predictor.predict({"inputs": text,
                             "parameters": {
                                 "max_length":45,
                                 "clean_up_tokenization_spaces":True
                                 }
                            })
print(outputs[0]['summary_text'])

 Bumblebee ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead.


In [57]:
predictor.delete_endpoint()

### Translation

In [58]:
predictor = setup_endpoint("Helsinki-NLP/opus-mt-en-de", "translation")

-----!

In [59]:
outputs = predictor.predict({"inputs": text,
                             "parameters": {
                                 "min_length":100,
                                 "clean_up_tokenization_spaces":True
                                 }
                            })
print(outputs[0]['translation_text'])

Sehr geehrter Amazon, letzte Woche habe ich eine Optimus Prime Action Figur aus Ihrem Online-Shop in Deutschland bestellt. Leider, als ich das Paket öffnete, entdeckte ich zu meinem Entsetzen, dass ich stattdessen eine Action Figur von Megatron geschickt worden war! Als lebenslanger Feind der Decepticons, Ich hoffe, Sie können mein Dilemma verstehen. Um das Problem zu lösen, Ich fordere einen Austausch von Megatron für die Optimus Prime Figur habe ich bestellt. Anbei sind Kopien meiner Aufzeichnungen über diesen Kauf. Ich erwarte, bald von Ihnen zu hören. Aufrichtig, Bumblebee.


In [61]:
predictor.delete_endpoint()

### Text Generation

In [36]:
predictor = setup_endpoint("gpt2", 'text-generation')

--------!

In [25]:
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response

outputs = predictor.predict({"inputs": prompt,
                             "parameters": {
                                 "max_length":200
                                 }
                            })
print(outputs[0]['generated_text'])

Dear Amazon, last week I ordered an Optimus Prime action figure from your online
store in Germany. Unfortunately, when I opened the package, I discovered to my
horror that I had been sent an action figure of Megatron instead! As a lifelong
enemy of the Decepticons, I hope you can understand my dilemma. To resolve the
issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered.
Enclosed are copies of my records concerning this purchase. I expect to hear
from you soon. Sincerely, Bumblebee.

Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up. Please review
this email to let us know if your order matches the item description you
received. Thank you! I have added my request to both my order description and my
customer service number in my note.

Customer service response:

Dear Bumblebee, your order was misprinted. We are sorry for the inconvenience
but please


In [ ]:
predictor.delete_endpoint()

## The Hugging Face Ecosystem

### The Hugging Face Hub

### Hugging Face Tokenizers

### Hugging Face Datasets

### Hugging Face Accelerate

## Main Challenges with Transformers

## Conclusion